# Airbus Uçak Görüntüleri: Gözetimsiz Öğrenme (Unsupervised Learning)

Bu notebook, Airbus uçak görüntüleri üzerinde gözetimsiz öğrenme yöntemleriyle analiz ve kümeleme işlemlerini kapsamaktadır. Her adımda Türkçe açıklamalar ve kodlar yer almaktadır.

---

Bu notebookta aşağıdaki ana başlıklar ve adımlar yer almaktadır:

1. Gerekli Kütüphanelerin Yüklenmesi
2. Veri Önişleme ve Görselleştirme (Unsupervised)
3. Veri Ölçekleme
4. Temel PCA ile Boyut İndirgeme ve Görselleştirme
5. K-Means Kümeleme ile Sınıflandırma
6. Kümeleme Sonuçlarının Görselleştirilmesi
7. Kümeleme Sonuçlarının Değerlendirilmesi (Silhouette Score, vs.)
8. Farklı Kümeleme Algoritmalarının Karşılaştırılması (Opsiyonel)

Her bölümün başında kısa açıklamalar ve kod blokları ile ilerleyebilirsiniz.

## 1. Gerekli Kütüphanelerin Yüklenmesi

NumPy, pandas, matplotlib, scikit-learn ve görselleştirme için gerekli diğer kütüphaneler yüklenir.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score

## 2. Veri Önişleme ve Görselleştirme (Unsupervised)

Görüntüler etiket olmadan yüklenir, yeniden boyutlandırılır ve örnek görseller gösterilir.

In [ ]:
# Görüntülerin yüklenmesi ve hazırlanması (etiketsiz)
veri_dizini = "Airbus/aircraft/detection dataset"
gorseller = []
for sinif_adi in os.listdir(veri_dizini):
    sinif_klasoru = os.path.join(veri_dizini, sinif_adi)
    for dosya_adi in os.listdir(sinif_klasoru):
        yol = os.path.join(sinif_klasoru, dosya_adi)
        img = cv2.imread(yol)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (64, 64))  # Daha hızlı işlem için küçük boyut
            gorseller.append(img)
gorseller = np.array(gorseller)

# Örnek görselleri göster
fig, axes = plt.subplots(1, 5, figsize=(15,3))
for i in range(5):
    axes[i].imshow(gorseller[i])
    axes[i].axis('off')
plt.suptitle("Örnek Görseller (Etiketsiz)")
plt.show()

## 3. Veri Ölçekleme

Görüntü verileri MinMaxScaler veya StandardScaler ile ölçeklenir. Bu adım, kümeleme ve boyut indirgeme algoritmalarının daha iyi çalışmasını sağlar.

In [ ]:
# Görüntüleri vektör haline getir ve ölçekle
gorseller_vektor = gorseller.reshape(gorseller.shape[0], -1)
scaler = StandardScaler()
gorseller_scaled = scaler.fit_transform(gorseller_vektor)
print(f"Ölçeklenmiş veri boyutu: {gorseller_scaled.shape}")

## 4. Temel PCA ile Boyut İndirgeme ve Görselleştirme

PCA ile veri 2 boyuta indirgenir ve scatter plot ile görselleştirilir.

In [ ]:
# PCA ile 2 boyuta indirgeme
pca = PCA(n_components=2)
gorseller_pca = pca.fit_transform(gorseller_scaled)

plt.figure(figsize=(8,6))
plt.scatter(gorseller_pca[:,0], gorseller_pca[:,1], alpha=0.5)
plt.title("PCA ile 2D Görselleştirme (Etiketsiz)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()

## 5. K-Means Kümeleme ile Sınıflandırma

KMeans algoritması ile görüntüler kümelere ayrılır ve küme etiketleri elde edilir.

In [ ]:
# KMeans ile kümeleme (ör: 3 küme)
kume_sayisi = 3
kmeans = KMeans(n_clusters=kume_sayisi, random_state=42)
kume_etiketleri = kmeans.fit_predict(gorseller_scaled)
print(f"Küme etiketleri: {np.unique(kume_etiketleri)}")

## 6. Kümeleme Sonuçlarının Görselleştirilmesi

Küme etiketlerine göre PCA ile indirgenmiş verinin scatter plot ile görselleştirilmesi.

In [ ]:
# Kümeleme sonuçlarının PCA düzleminde görselleştirilmesi
plt.figure(figsize=(8,6))
for k in np.unique(kume_etiketleri):
    plt.scatter(gorseller_pca[kume_etiketleri==k,0], gorseller_pca[kume_etiketleri==k,1], label=f'Küme {k}')
plt.title("KMeans Kümeleme Sonuçları (PCA 2D)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.legend()
plt.show()

## 7. Kümeleme Sonuçlarının Değerlendirilmesi (Silhouette Score, vs.)

Silhouette Score ve Davies-Bouldin Index gibi metriklerle kümeleme başarısı değerlendirilir.

In [ ]:
# Kümeleme başarısı metrikleri
sil_score = silhouette_score(gorseller_scaled, kume_etiketleri)
db_score = davies_bouldin_score(gorseller_scaled, kume_etiketleri)
print(f"Silhouette Score: {sil_score:.3f}")
print(f"Davies-Bouldin Index: {db_score:.3f}")

## 8. Farklı Kümeleme Algoritmalarının Karşılaştırılması (Opsiyonel)

Agglomerative Clustering ve DBSCAN gibi alternatif algoritmalarla sonuçlar karşılaştırılır ve görselleştirilir.

In [ ]:
# Agglomerative Clustering
agg = AgglomerativeClustering(n_clusters=kume_sayisi)
agg_labels = agg.fit_predict(gorseller_scaled)

plt.figure(figsize=(8,6))
for k in np.unique(agg_labels):
    plt.scatter(gorseller_pca[agg_labels==k,0], gorseller_pca[agg_labels==k,1], label=f'Agglo Küme {k}')
plt.title("Agglomerative Clustering Sonuçları (PCA 2D)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.legend()
plt.show()

# DBSCAN
dbscan = DBSCAN(eps=5, min_samples=5)
db_labels = dbscan.fit_predict(gorseller_scaled)

plt.figure(figsize=(8,6))
for k in np.unique(db_labels):
    plt.scatter(gorseller_pca[db_labels==k,0], gorseller_pca[db_labels==k,1], label=f'DBSCAN Küme {k}')
plt.title("DBSCAN Sonuçları (PCA 2D)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.legend()
plt.show()